In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.ndimage.morphology import distance_transform_edt
import keras
listOut=[]
listA=[]
listB=[]
for i in range(1000):
    X=np.zeros([28,28])
    a=np.random.randint(28)
    X[a,1]=1
    b=np.random.randint(28)
    #b=14
    X[b,27]=1
    A=distance_transform_edt(X==0)<10
    listOut.append(A)
    listA.append(a)
    listB.append(b)
#plt.imshow(A>10)
#plt.show()
plt.imshow(A)
plt.show()
plt.imshow(listOut[10])
plt.show()

In [ ]:
plt.plot(listA,listB,'.')
plt.show()

In [ ]:
print(np.array(listOut).shape)
listOut=np.expand_dims(np.array(listOut),axis=3)*1
print(np.array(listOut).shape)
listOut = listOut.astype('float32')

In [ ]:
from __future__ import print_function

import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import norm

from keras.layers import Input, Dense, Lambda, Flatten, Reshape
from keras.layers import Conv2D, Conv2DTranspose
from keras.models import Model
from keras import backend as K
from keras import metrics
from keras.datasets import mnist

# input image dimensions
img_rows, img_cols, img_chns = 28, 28, 1
# number of convolutional filters to use
filters = 16
# convolution kernel size
num_conv = 3

batch_size = 12
if K.image_data_format() == 'channels_first':
    original_img_size = (img_chns, img_rows, img_cols)
else:
    original_img_size = (img_rows, img_cols, img_chns)
latent_dim = 2
intermediate_dim = 32
epsilon_std = 1.0
epochs = 50




In [ ]:
#Decoder Model

output_shape = (batch_size, 14, 14, filters)
xdec = Input(shape=(latent_dim,),name='decoder_input')
dec_hid = Dense(intermediate_dim, activation='relu')(xdec)
dec_upsample = Dense(filters * 14 * 14, activation='relu')(dec_hid)
dec_reshape = Reshape(output_shape[1:])(dec_upsample)
dec_deconv_1 = Conv2DTranspose(filters,kernel_size=(3,3),padding='same',strides=(2,2),activation='relu')(dec_reshape)
dec_deconv_1 = Conv2D(1,kernel_size=(1,1),padding='same',strides=(1,1),activation='relu')(dec_deconv_1)
Decoder= Model(xdec, outputs=dec_deconv_1)
Decoder.summary()



In [ ]:
#Encoder for Variational AutoEncoder 

x = Input(shape=original_img_size,name='main_input')
conv_1 = Conv2D(img_chns,kernel_size=(2, 2),padding='same', activation='relu')(x)
conv_2 = Conv2D(filters,kernel_size=(2, 2),padding='same', activation='relu',strides=(2, 2))(conv_1)
conv_3 = Conv2D(filters,kernel_size=num_conv,padding='same', activation='relu',strides=1)(conv_2)
conv_4 = Conv2D(filters,kernel_size=num_conv,padding='same', activation='relu',strides=1)(conv_3)
flat = Flatten()(conv_4)
hidden = Dense(latent_dim, activation='relu')(flat)

encoder=Model(x,outputs=hidden)
encoder.compile(optimizer='adam', loss='mse')
encoder.summary()

ae=Model(x, outputs=Decoder(hidden))
ae.summary()



In [ ]:
ae.compile(optimizer='adam', loss='mse')
ae.summary()

ae.fit(x=listOut, y=listOut,shuffle=True,epochs=30,batch_size=batch_size)

In [ ]:
proj=encoder.predict(listOut)

In [ ]:
import matplotlib.pyplot as plt
plt.plot(proj[:,0],proj[:,1],'.')
plt.xlabel('Latent Variable 1')
plt.ylabel('Latent Variable 2')
plt.show()

plt.plot(proj[:,0],listA,'.')
plt.xlabel('Latent Variable 1')
plt.ylabel('First Coordinate on Manifold')
plt.show()
plt.plot(proj[:,1],listB,'.')
plt.xlabel('Latent Variable 2')
plt.ylabel('Second Coordinate on Manifold')
plt.show()

In [ ]:
from scipy.stats import norm
n = 25
digit_size = 28
figure = np.zeros((digit_size * n, digit_size * n))
grid_x = np.linspace(0, 25, n)
grid_y = np.linspace(0, 25, n)
for i, yi in enumerate(grid_x):
    for j, xi in enumerate(grid_y):
        z_sample = np.array([[xi, yi]])
        z_sample = np.tile(z_sample, batch_size).reshape(batch_size, 2)
        x_decoded = Decoder.predict(z_sample, batch_size=batch_size)
        digit = x_decoded[0].reshape(digit_size, digit_size)
        figure[i * digit_size: (i + 1) * digit_size,j * digit_size: (j + 1) * digit_size] = digit
plt.figure(figsize=(10, 10))
plt.imshow(figure>0, cmap='Greys_r')
plt.show()

#plt.figure(figsize=(10, 10))
#plt.imshow(figure>0.1, cmap='Greys_r')
#plt.show()

In [ ]:
from keras import losses
class_input = K.zeros(shape=(1,28,28,1))
loss = losses.mse(class_input, ae.output)
im_input = ae.input # This is the input image
grads = K.gradients(loss, im_input)[0] # the signs will acheive the desired effect
grads /= K.maximum(K.mean(K.abs(grads)), K.epsilon()) # Normalize for numerical stability
outputs = [grads]
adv_noise = K.function([im_input, class_input], outputs)

In [ ]:
original_img_size

In [ ]:
imin=listOut[0:1,:,:,:]
res=adv_noise([np.expand_dims(imin[0,:,:,:],axis=0),np.expand_dims(imin[0,:,:,:],axis=0)])

In [ ]:
plt.imshow(listOut[0,:,:,0])
plt.show()
plt.imshow(res[0][0,:,:,0])
plt.show()